In [1]:
import glob, re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


path='/home/marinjl/work/PtFige-Lille.NEW/RESULTS/'
#path='/home/marinjl/work/PtFige-Lyon.NEW/RESULTS/'
#path='/home/marinjl/work/PtFige-Marseille.NEW/RESULTS/'
device = 'busloads'


def get_devices(path, device):
    files = glob.glob(path + '/*Astre.csv')
    return [x.split('-')[1].split(device +'_')[1] for x in files]


In [2]:
# returns 'vini', 'v_fin', 't_trans', 'max_trans', 'min_trans'
def analyze_dyn(df,vv, eps):
    x = df[vv].values
    t = df['time'].values

    dd = np.where(np.abs(np.diff(x))> eps)[0]
    dd = dd[np.where(dd>3)]

    if(len(dd) == 0):
        return [x[0],x[0],0,0,0]

    ll = len(dd)
    xin = x[dd[0]]
    x0 = x[dd[0]+1]
    ifin = dd[ll-1] + 1
    xfin = x[ifin]
    ttran = t[ifin]-300
    xmax = max(x[dd]) - x0
    xmin = min(x[dd]) - x0

    return([xin, xfin, ttran, xmax, xmin])

def get_dfs(device,dv):
    f_ast = path + device + '_' + dv +'-Astre.csv'
    f_dyn = path + device + '_' + dv +'-Dynawo.csv'

    df_ast = pd.read_csv(f_ast, sep =';')
    df_dyn = pd.read_csv(f_dyn, sep =';')
    df_dyn = df_dyn.iloc[:, :-1]   ###df_dyn.drop('Unnamed: 24',axis=1)
    df_dyn.columns=df_ast.columns

    # tfin start at 4000 s.
    df_dyn['time'] = df_dyn.time - 4000
    return df_ast,df_dyn

In [3]:
devices = get_devices(path,device)
cnames = ['v_ini','v_fin','t_trans','max_trans','min_trans']
eps = 1e-6

all_ast = pd.DataFrame()
all_dyn = pd.DataFrame()

dataiter = tqdm(devices)

for dv in dataiter:
    
    #print(dv)
    
    df_ast,df_dyn = get_dfs(device,dv)
    
    
    # Variables match by order, not name
    vars_ast = df_ast.columns[1:]
    vars_dyn = df_dyn.columns[1:] 

    res_ast =[analyze_dyn(df_ast,x,eps) for x in vars_ast]
    res_dyn =[analyze_dyn(df_dyn,x,eps) for x in vars_dyn]

    
    dd_ast=pd.DataFrame(data = res_ast, columns = cnames)
    dd_dyn=pd.DataFrame(data = res_dyn, columns = cnames)

    dd_ast['vars'] = vars_ast
    dd_dyn['vars'] = vars_dyn

    dd_ast['dev'] = dv
    dd_dyn['dev'] = dv
    
    all_ast = all_ast.append(dd_ast)
    all_dyn = all_dyn.append(dd_dyn)


all_ast.to_csv('all_astre.csv')
all_dyn.to_csv('all_dynaw.csv')

In [4]:
delta = all_ast[['dev','vars']].copy(deep = True)

delta['dV_ast'] = all_ast.v_fin-all_ast.v_ini
delta['dV_dyn'] = all_dyn.v_fin-all_dyn.v_ini
delta['dT_ast'] = all_ast.max_trans-all_ast.min_trans
delta['dT_dyn'] = all_dyn.max_trans-all_dyn.min_trans
delta['lT_ast'] = all_ast.t_trans
delta['lT_dyn'] = all_dyn.t_trans

delta.to_csv('delta_ast_dyn.csv')

delta

,dev,vars,dV_ast,dV_dyn,dT_ast,dT_dyn,lT_ast,lT_dyn
0,.FARCP51,RST_WARANP7_U_IMPIN_value,0.000000,0.000000,0.000000,0.000000,0,0.0
1,.FARCP51,RST_WARANP7_levelK_value,0.000000,0.000000,0.000000,0.000000,0,0.0
2,.FARCP51,DM_GRAV5T 1_tfo_generator_PGen,0.000000,0.000000,0.000000,0.000000,0,0.0
3,.FARCP51,DM_GRAV5T 1_tfo_generator_QGen,0.000000,0.000000,0.000000,0.000000,0,0.0
4,.FARCP51,DM_GRAV5T 3_tfo_generator_PGen,0.000000,0.000000,0.000000,0.000000,0,0.0
5,.FARCP51,DM_GRAV5T 3_tfo_generator_QGen,0.000000,0.000000,0.000000,0.000000,0,0.0
6,.FARCP51,DM_GRAV5T 4_tfo_generator_PGen,0.000000,0.000000,0.000000,0.000000,0,0.0
7,.FARCP51,DM_GRAV5T 4_tfo_generator_QGen,0.000000,0.000000,0.000000,0.000000,0,0.0
8,.FARCP51,DM_GRAV5T 5_tfo_generator_PGen,0.000000,0.000000,0.000000,0.000000,0,0.0
9,.FARCP51,DM_GRAV5T 5_tfo_generator_QGen,0.000000,0.000000,0.000000,0.000000,0,0.0


In [5]:
import plotly.graph_objects as go
from ipywidgets import widgets

mask_n=['NETWORK' in x for x in all_dyn.vars]
df = delta[mask_n]

var = widgets.Dropdown(
    options=list(['dV','dT','lT']),
    value='dV',
    description='Variable:   ',
    
)

mask = widgets.Dropdown(
    options=list(['NETWORK','U_IMPIN','levelK','PGen','QGen']),
    value='NETWORK',
    description='Monitor',
)

trace = go.Scatter(x=df['dV_ast'],
                   y=df['dV_dyn'],
                   mode='markers',
                   marker_color=df['lT_ast'],
                   marker_size=(df.dT_ast-min(df.dT_ast))/(max(df.dT_ast)-min(df.dT_ast))*100,
                   text=df['dev'])

g = go.FigureWidget(data=trace,
                    layout=go.Layout(
                        title=dict(
                            text='Astre vs Dynawo'
                        ),
                
                    ))

In [6]:
def response(change):
    mask_=[mask.value in x for x in all_dyn.vars]
    df = delta[mask_]
    with g.batch_update():
            g.data[0].x = df[var.value + '_ast']
            g.data[0].y = df[var.value + '_dyn']
            g.data[0].marker.color = df.lT_ast
            g.data[0].marker.size = (df.dT_ast-min(df.dT_ast))/(max(df.dT_ast)-min(df.dT_ast))*50
            g.data[0].text = df.dev
            
            g.layout.xaxis.title = var.value + 'Astre'
            g.layout.yaxis.title = var.value + 'Dynawo'
            
var.observe(response, names="value")
mask.observe(response, names="value")

In [7]:
container = widgets.HBox([mask, var])
widgets.VBox([container,g])
              


In [8]:
dv0 = 'CRENEP61' # Lille
#dv0 = 'CORNIP31' # Lyon
#dv0 = 'ALBERP62' # Marseille

dev = widgets.Dropdown(
    options=list(delta['dev'].unique()),
    value= dv0,
    description='Device'
)


df_ast,df_dyn = get_dfs(device,dv0)

# Variables match by order, not name
vars_ast = df_ast.columns[1:]
vars_dyn = df_dyn.columns[1:] 


var0 = vars_ast[0]

var2 = widgets.Dropdown(
    options=vars_ast,
    value=var0,
    description='Variable:   ',
    
)


trace1 = go.Scatter(x=df_ast['time'],
                   y=df_ast[var0],
                   mode='lines+markers',
                   marker_color='black'
                   )

trace2 = go.Scatter(x=df_dyn['time'],
                   y=df_dyn[var0]-df_dyn[var0][0]+df_ast[var0][1],
                   mode='lines',
                   marker_color='red'
                   )



g2 = go.FigureWidget(data=[trace1,trace2],
                    layout=go.Layout(
                        title=dict(
                            text='Astre vs Dynawo'
                        ),
                
                    ))



In [9]:
def response2(change):
    df_ast,df_dyn = get_dfs(device,dev.value)
    vars_ast = df_ast.columns[1:] 
    
    var2.options=vars_ast
    var0 = vars_ast[0]
    var2.value = var0
    
    with g2.batch_update():
            g2.data[0].x = df_ast['time']
            g2.data[0].y = df_ast[var0]
            g2.data[1].x = df_dyn['time']
            g2.data[1].y = df_dyn[var0]-df_dyn[var0][0]+df_ast[var0][1]
            
def response3(change):
    df_ast,df_dyn = get_dfs(device,dev.value) 
    
    with g2.batch_update():
            g2.data[0].y = df_ast[var2.value]
            g2.data[1].y = df_dyn[var2.value]-df_dyn[var2.value][0]+df_ast[var2.value][1]
            
            
            
dev.observe(response2, names="value")
var2.observe(response3, names="value")

In [10]:
container2 = widgets.HBox([dev, var2])
widgets.VBox([container2,g2])
              